# Magnitudes Físicas

## Incertidumbre

El módulo [uncertainties](http://pythonhosted.org/uncertainties/) automatiza la propagación de errores:

In [ ]:
from uncertainties import ufloat

In [ ]:
x = ufloat(1.345387, 0.004)

In [ ]:
x

In [ ]:
5*x**2 + 1

Definimos funciones para mostrar 2 cifras significativas en la incertidumbre, siguiendo las recomendaciones establecidas.

In [ ]:
def pretty(x):
    print('{:.2uS}'.format(x))

def prettyl(x):
    print('{:.2uP}'.format(x))

In [ ]:
prettyl( x )

In [ ]:
pretty( x )

In [ ]:
pretty( 5*x**2 +1 )

Este módulo tiene en cuenta la covarianza entre variables para manejar correctamente sus posibles dependencias. En el siguiente ejemplo `y` y `z` son aparentemente iguales, en magnitud e incertidumbre, pero se comportan de forma distinta al operar con `x`. Aquí `y` es independiente de `x`, mientras que `z` es una fracción de la misma magnitud `x`.

In [ ]:
x = ufloat(2, 0.02)
y = ufloat(1, 0.01)
z = x/2

prettyl(x)
prettyl(y)
prettyl(z)
prettyl( x + y )
prettyl( x - y )
prettyl( x + z )
prettyl( x - z )

La incertidumbre se puede introducir de forma más bonita con la siguiente función auxiliar:

In [ ]:
# Python admite símbolos unicode

# Ctrl-May-U + 025b + espacio
def ɛ(v):
    return v*ufloat(0,1)

Sirve para "añadir" un error o incertidumbre absoluta a una magnitud.

In [ ]:
x = 1.345387 + ɛ(0.004)

pretty( x )

In [ ]:
pretty( x**2 )

Cada magnitud debe tener su propio ɛ para que los errores sean independientes.

Es más práctico usar la siguiente función, que crea cómodamente valores con un error relativo expresado en %.

In [ ]:
def mag(v,ep):
    return v * (1 + ɛ(ep/100))

In [ ]:
mag(25,3)

In [ ]:
# otro símbolo útil

import numpy as np

# 03c0
π = np.pi

In [ ]:
r = mag(5 , 2)

pretty( π * r**2 )

Un pequeño inconveniente es que las funciones matemáticas hay que importarlas del módulo `uncertainties`.

In [ ]:
from uncertainties.umath import sqrt

pretty( sqrt (1.345387 + ɛ(0.004)) )

Por suerte los operadores funcionan automáticamente.

In [ ]:
pretty( (1.345387 + ɛ(0.004))**(1/2) )

## Unidades físicas

Hay [varios paquetes](https://socialcompare.com/en/comparison/python-units-quantities-packages-383avix4) para manejar magnitudes físicas, una posibilidad es [pint](http://pint.readthedocs.io/en/0.8/tutorial.html).

In [ ]:
# (necesario solo en jupyterlite)
import sys
if 'pyodide' in sys.modules:
    %pip install pint

In [ ]:
from pint import UnitRegistry
u = UnitRegistry()

In [ ]:
(3*u.meter / (2*u.millisecond)).to(u.kilometer/u.hour)

Podemos combinar de forma natural las unidades físicas y la incertidumbre:

In [ ]:
2*u.kilogram + ɛ(3*u.gram)

In [ ]:
mag( 2*u.meter/u.sec , 5)

Como ejemplo, definimos una función normal y corriente de Python para calcular la energía cinética de un cuerpo.

In [ ]:
def kin(m,v):
    return 1/2*m*v**2

In [ ]:
kin(2,3)

Y la evaluamos con magnitudes con unidades físicas e incertidumbre.

In [ ]:
m = 2*u.kilogram + ɛ(3*u.gram)

v = mag( 3*u.kilometer/u.hour , 5 )

Ec = kin(m, v)

pretty(Ec)

El resultado se puede expresar en cualquier otra unidad:

In [ ]:
pretty(Ec.to(u.joule))

In [ ]:
pretty(Ec.to(u.calorie))

In [ ]:
pretty(Ec.to(u.kilowatt*u.hour))

In [ ]:
pretty(Ec.to(u.electron_volt))

## Caso de estudio: péndulo de [Kater](https://en.wikipedia.org/wiki/Kater%27s_pendulum)

$$g = \frac{8\pi^2}{\frac{T_1^2+T_2^2}{H}+\frac{T_1^2-T_2^2}{h_1 -h_2}}$$

In [ ]:
def estimag(T1,T2,H,h1):
    h2 = H-h1
    return 8*π**2/((T1**2+T2**2)/H + (T1**2-T2**2)/(h1-h2))

In [ ]:
estimag(2*u.sec, 2.003*u.sec, 1*u.meter, 0.7*u.meter)

In [ ]:
g = estimag(2*u.sec + ɛ(20/400*u.centisec), 2.003*u.sec, 1*u.meter, 0.7*u.meter)
pretty(g)

In [ ]:
g = estimag(2    *u.sec + ɛ(20/400*u.centisec),
            2.003*u.sec + ɛ(20/400*u.centisec),
            1*u.meter,
            0.7*u.meter + ɛ(0*u.millimeter) )
pretty(g)

In [ ]:
g = estimag(2    *u.sec + ɛ(20/400*u.centisec),
            2.003*u.sec + ɛ(20/400*u.centisec),
            1*u.meter,
            0.7*u.meter + ɛ(10*u.millimeter) )
pretty(g)

## Constantes físicas

Están disponibles en el módulo [scipy.constants](https://docs.scipy.org/doc/scipy-0.19.0/reference/constants.html).

In [ ]:
import scipy.constants as const

In [ ]:
const.Planck

In [ ]:
const.physical_constants['deuteron mass']

Podemos convertirlas al formato anterior, con incertidumbre y unidades:

In [ ]:
def konst(name):
    v,unit,i = const.physical_constants[name]
    return ufloat(v,i) * u.parse_expression(unit)

In [ ]:
pretty(konst('electron mass'))

In [ ]:
pretty(konst('Boltzmann constant'))

In [ ]:
const.epsilon_0

In [ ]:
pretty(konst('electric constant'))

In [ ]:
const.find('avog')

In [ ]:
pretty(konst(_[0]))

In [ ]:
const.find('lig')

In [ ]:
pretty(konst(_[0]).to(u.cm / u.nanosec))

## Caso de estudio: fuerza eléctrica en átomo de H.

$$F = \frac{1}{4\pi\epsilon_0}\frac{q Q}{r^2}$$

In [ ]:
r = konst('Bohr radius')
pretty(r.to(u.angstrom))

e = konst('elementary charge')
pretty(e)

eps_0 = konst('electric constant')
pretty(eps_0)

F = 1/4/π/eps_0 * e * e / r**2

pretty(F)
pretty(F.to(u.newton))

Velocidad clásica en la órbita:

$$ F = \frac{m v^2}{r} $$ 

$$ v = \sqrt{\frac{r F}{m}} $$

In [ ]:
me = konst('electron mass')

v = (r*F/me)**(1/2)
pretty(v.to(u.km/u.hour))
pretty(v.to_base_units())

c = konst('speed of light in vacuum')
pretty((v/c).to_base_units())

In [ ]:
v.error_components()

$$ v = \sqrt{\frac{r \frac{1}{4\pi\epsilon_0}\frac{q Q}{r^2}}{m}} = \sqrt{\frac{q Q}{4\pi\epsilon_0 m r}} $$

In [ ]:
v = (e * e / 4/π/eps_0 / me/ r)**(1/2)
pretty((v/c).to_base_units())

In [ ]:
v.error_components()

In [ ]:
eqm = abs(konst('electron charge to mass quotient'))

pretty(eqm)
pretty(e / me)

In [ ]:
v = (eqm * e / 4/π/eps_0 / r)**(1/2)
pretty((v/c).to_base_units())

## Caso de estudio: comparación con Monte-Carlo.

In [ ]:
er_F = 0.05; er_M = 0.03; er_m = 0.01; er_d = .05
mu_F = 1e-8;
mu_M = 3;
mu_m = 0.05;
mu_d = 0.032;

F = mag(mu_F,100*er_F)
M = mag(mu_M,100*er_M)
m = mag(mu_m,100*er_m)
d = mag(mu_d,100*er_d)

In [ ]:
pretty(F)
pretty(M)
pretty(m)
pretty(d)

In [ ]:
G = d**2 * F / ( M * m )

pretty(G)
prettyl(G)

Consistente con Monte-Carlo:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
sigma_F = er_F * mu_F
sigma_M = er_M * mu_M
sigma_m = er_m * mu_m
sigma_d = er_d * mu_d

N = 100000
F = mu_F + sigma_F * np.random.randn( N )
M = mu_M + sigma_M * np.random.randn( N )
m = mu_m + sigma_m * np.random.randn( N )
d = mu_d + sigma_d * np.random.randn( N )

G = d**2 * F / ( M * m )

plt.hist( G, 40, density = 1 );

In [ ]:
np.mean(G)

In [ ]:
np.std(G)

## Caso de estudio: período de una órbita

In [ ]:
# Agrupamos aquí las utilidades de este tema

import numpy as np

from pint import UnitRegistry
u = UnitRegistry()

from uncertainties import ufloat

def pretty(x):
    print('{:.2uS}'.format(x))

def prettyl(x):
    print('{:.2uP}'.format(x))

def ɛ(v):
    return v*ufloat(0,1)

def mag(v,ep):
    return v * (1 + ɛ(ep/100))

Partiendo de la distancia y la velocidad en un instante calculamos el período de la órbita propagando la incertidumbre. Usamos datos parecidos a los de la órbita de la Luna.

Velocidad:

In [ ]:
v = mag(1 * u.kilometer / u.second, 3)
pretty(v)

Distancia:

In [ ]:
r = mag(384000 * u.kilometer, 1)
pretty(r)

Parámetro gravitacional de La Tierra.

In [ ]:
# GM Tierra
mu = 3.986E14 *u.meter**3 / u.second**2

Energía de la órbita:

In [ ]:
e = v**2/2 - mu/r

print(e)
pretty(e.to(u.joule/u.kilogram))

Semieje mayor:

In [ ]:
a = -mu/2/e

print(a)
pretty(a.to(u.meter))

Período:

In [ ]:
T = 2*np.pi*(a**3/mu)**(1/2)

print(T)
prettyl(T.to(u.day))
print(f'{100*T.std_dev / T.nominal_value:.0f}%')

El error en la velocidad se amplifica bastante.